### FE621 - Homework #2

**Author**: Sid Bhatia

**Date**: March 4th, 2023

**Pledge**: I pledge my honor that I have abided by the Stevens Honor System.

**Professor**: Sveinn Olafsson

**TA**: Dong Woo Kim

#### Problem #1

a. Provide the formulas for the parameters $p, u, d$ of a **Cox-Ross-Rubinstein (CRR) binomial tree** that approximates the dynamics of a stock following *geometric Brownian motion* (*GBM*) with drift $r$ and volatility $\sigma$.

The Cox-Ross-Rubinstein (CRR) binomial tree model is a popular method for pricing European and American options. This model approximates the dynamics of a stock following a geometric Brownian motion (GBM) by discretizing the stock price movements over time into an up or down movement at each step of the tree. The parameters $p, u, d$ are crucial for constructing the binomial tree. They represent the probability of an up move, the up move factor, and the down move factor, respectively. The formulas for these parameters are derived from the characteristics of the geometric Brownian motion, which is described by a drift rate $r$ (risk-free interest rate) and a volatility $\sigma$. The time to maturity of the option is divided into $n$ equal intervals of length $\Delta t$.

The formulas for the parameters are as follows:

- **Up move factor ($u$):**
  $$
  u = e^{\sigma \sqrt{\Delta t}}
  $$
  This factor represents the multiplicative increase in the stock price if it moves up.

- **Down move factor ($d$):**
  $$
  d = e^{-\sigma \sqrt{\Delta t}}
  $$
  This factor represents the multiplicative decrease in the stock price if it moves down. It's worth noting that $d = \frac{1}{u}$, ensuring that the tree is recombining.

- **Risk-neutral probability ($p$):**
  $$
  p = \frac{e^{r \Delta t} - d}{u - d}
  $$
  This probability is used to calculate the expected value of the option at each node in a risk-neutral world, where all investors are indifferent to risk. It reflects the probability of the stock price moving up in the next time step.

These parameters ensure that the binomial model converges to the continuous-time geometric Brownian motion model as the number of steps $n$ increases and $\Delta t$ becomes very small.

b. Write code that takes $S_0, \sigma, r, K, T$ and $N$ as inputs and uses an $N$-step CCR binomial tree to compute the prices of **call** and **put options** in the **Black-Scholes model**.

In [2]:
from typing import Tuple
import numpy as np

def CRR_parameters(sigma: float, r: float, T: float, N: int) -> tuple:
    """
    Calculate the Cox-Ross-Rubinstein binomial model parameters.

    Parameters:
    - sigma: Volatility of the underlying asset.
    - r: Risk-free interest rate.
    - T: Time to maturity.
    - N: Number of time steps.

    Returns:
    - A tuple (u, d, p) where:
        u is the up factor,
        d is the down factor,
        p is the risk-neutral probability.
    """
    dt = T / N  # Delta t
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    return (u, d, p)

def binomial_tree_option_price(S0: float, K: float, T: float, sigma: float, r: float, N: int, option_type: str) -> float:
    """
    Calculate the European option price using the Cox-Ross-Rubinstein binomial tree.

    Parameters:
    - S0: Initial stock price.
    - K: Strike price.
    - T: Time to maturity.
    - sigma: Volatility of the underlying asset.
    - r: Risk-free interest rate.
    - N: Number of time steps.
    - option_type: Type of the option ('call' or 'put').

    Returns:
    - Price of the option.
    """
    u, d, p = CRR_parameters(sigma, r, T, N)
    dt = T / N
    discount = np.exp(-r * dt)
    
    # Initialize asset prices at maturity.
    S = np.array([S0 * (u**j) * (d**(N - j)) for j in range(N + 1)])
    
    # Initialize option values at maturity.
    if option_type == 'call':
        V = np.maximum(S - K, 0)
    else:  # put option
        V = np.maximum(K - S, 0)
    
    # Backward induction
    for i in range(N - 1, -1, -1):
        V = discount * (p * V[1:i+2] + (1 - p) * V[:i+1])
    
    return V[0]

c. Analyze the **convergence** of binomial tree option prices to Black-Scholes option prices as the number of time steps gets larger. Specifically, for both call and put options, plot the Black-Scholes price $P^{BS}$ and the $N$-step binomial tree price $P_N^{\text{Tree}}$ as a function of $N$. Also, plot the absolute relative pricing error $\frac{|P_N^{\text{Tree}} - P^{BS}|}{P^{BS}}$ as a function of $N$.

Comment on your findings. Does the error decrease smoothly to zero as $N$ grows? How large does $N$ have to be (i.e., how small does the step size $\Delta t$ for satisfactory convergence)?

Use parameters $S_0 = 100, K = 90, T = 0.5, \sigma = 0.2,$ and $r = 0.04$.

#### Problem #2

a. **Prove** that the log-return of the stock price in the Black-Scholes model satisfies

$$\log (\frac{S_T}{S_0}) = (r - \frac{\sigma^2}{2})T + \sigma W_T,$$

where $W_T \sim \mathcal{N}(0, T)$. Plot the PDF of the distribution of the log-return. 